In [1]:
pip install galois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 32.6 MB/s eta 0:00:00


In [2]:
# Needed libraries
import secrets
import math
import numpy as np
import hashlib
import galois

# Generate private seed

In [3]:
#Generate Private Seed
def generate_private_seed()->bytes:
  return secrets.token_bytes(32)

# SHAKE A.K.A function H (Squeeze public_seed and T)

## SHAKE using hashlib

In [5]:
def create_private_sponge(private_seed:bytes,v:int,m:int,lvl)->bytes:
  # USE SHAKE
  if(lvl==1):
    h_shake = hashlib.shake_128(private_seed)
  else:
    h_shake = hashlib.shake_256(private_seed)
  # Private Sponge
  return h_shake.digest(32+math.ceil(m/8)*v)

#Extract public seed

In [6]:
def get_public_seed(private_sponge:bytes)->bytes:
  return private_sponge[:32]

#Extract T

In [7]:
def int8_to_bits(intByte:int)->list:
  """Get the first 8 bits of an integer as an array.

  The function takes one integer byte, extracts it's bits
  and returns an array containing the bits starting from
  the most significant bit.

  Parameters
  ----------
  intByte : int
    The byte, labeled as an int by python.

  Returns
  -------
  list
      An array with the first 8 bits of the integer
      starting from the most significant one.
  """
  bits_data = bin(intByte)[2:]
  return [int(i) for i in bits_data.zfill(8 * ((len(bits_data) + 7) // 8))]

In [8]:
def tMatrixRowGen(byteString:bytes,neededBits:int)->list:
  """Generate one row of the T matrix.

  The function takes one byte-string and extracts the specified
  ammount of bits (neededBits) to fill one row. When the ammount
  of bits doesn't fit completely in a byte, the most significant
  bits of the last byte are ignored.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the row.
  neededBits : int
    The ammount of bits a row needs to have.

  Returns
  -------
  list
      An array representing one row of the T matrix
  """
  row = []
  while(neededBits>8):
    row+=int8_to_bits(byteString[0])
    byteString = byteString[1:]
    neededBits = neededBits-8
  if(neededBits>0):
    row+=int8_to_bits(byteString[0])[-neededBits:]
  return row

def tMatrixGenerator(byteString:bytes,v:int,m:int)->np.ndarray:
  """Generate the T matrix.

  The function takes one byte-string and generates v rows
  each one with m bits extracted sequentially from a byteString
  using the tMatrixRowGen() function.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.
  v : int
    The ammount rows of the matrix.
  m : int
    The ammount columns of the matrix.

  Returns
  -------
  list
      An array representing the T matrix
  """
  mdivided8 =math.ceil(m/8)
  matrix = []
  for i in range(v):
    matrix.append(tMatrixRowGen(byteString,m))
    byteString = byteString[mdivided8:]
  return np.array(matrix)

# SHAKE (squeeze public map)

In [9]:
def subMatrixGenerator(byteString:bytes)->np.ndarray:
  """Generate a part of the L or Q1 matrix.

  The function takes one byte-string and returns one 16 row part
  of the L or Q1 matrix. It must be noted that the rows are generated as columns
  at first and then are transposed.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.

  Returns
  -------
  np.ndarray
      A matrix
  """

  matrix = [int8_to_bits(byteString[i*2])+int8_to_bits(byteString[i*2+1]) for i in range(len(byteString)//2)]

  return np.array(matrix).T

## Obtain C, L, and $$Q_1$$

In [12]:
def G(v:int,m:int,lvl:int,public_seed:bytes)->tuple[np.ndarray,np.ndarray,np.ndarray]:
  n = m+v
  g_equation = 2+2*n+v*(v+1)+2*v*m
  mdivided16 = math.ceil(m/16)
  #first g function iteration
  if(lvl==1):
    g_shake = hashlib.shake_128(public_seed+bytes.fromhex('00'))
  else:
    g_shake = hashlib.shake_256(public_seed+bytes.fromhex('00'))
  public_sponge = g_shake.digest(g_equation)
  C_base=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
  public_sponge = public_sponge[2:]
  # L and Q1 have more than one dimension, so they are initilized differently
  L_base = subMatrixGenerator(public_sponge[:2*n])
  public_sponge = public_sponge[2*n:]
  Q1_base = subMatrixGenerator(public_sponge)

  #following g function iterations
  if(lvl==1):
    for i in range(1,mdivided16):
      g_shake = hashlib.shake_128(public_seed+bytes.fromhex(f'0{i}'))
      public_sponge = g_shake.digest(g_equation)
      C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
      public_sponge = public_sponge[2:]
      L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
      public_sponge = public_sponge[2*n:]
      Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))
  else:
    for i in range(1,mdivided16):
      g_shake = hashlib.shake_256(public_seed+bytes.fromhex(f'0{i}'))
      public_sponge = g_shake.digest(g_equation)
      C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
      public_sponge = public_sponge[2:]
      L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
      public_sponge = public_sponge[2*n:]
      Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))

  # obtaining C: we take the m bits (starting from the last generated bit) we
  # need fromthe total of bits generated
  C = C_base[-m:]
  C = np.array(C)
  # obtaining L: We discard the upper rows to get the specified dimensions
  L = L_base[-m:]
  # obtaining Q_1: We discard the upper row to get the specified dimensions
  Q1=Q1_base[-m:]
  return C,L,Q1


## Obtaining $$Q_2$$

In [13]:
def findPk1(v:int,m:int,k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk1 algorithm of the LUOV specification.

  Parameters
  ----------
  v : int
    The v parameter of the LUOV specification.
  m : int
    The m parameter of the LUOV specification.
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-v matrix representing the part of pk that is
      quadratic in the vinegar variables.
  """
  Pk1 = np.zeros([v,v],dtype=np.int8)
  column = 0
  for i in range(v):
    for j in range(i,v):
      Pk1[i,j]= q1[k,column]
      column+=1
    column+=m
  return Pk1

In [14]:
def findPk2(v:int,m:int,k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk2 algorithm of the LUOV specification.

  Parameters
  ----------
  v : int
    The v parameter of the LUOV specification.
  m : int
    The m parameter of the LUOV specification.
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-m matrix representing the part of pk that is bilinear in the
      vinegar variables and the oil variables.
  """
  Pk2 = np.zeros([v,m],dtype=np.int8)
  column = 0
  for i in range(v):
    column += v-i
    for j in range(m):
      Pk2[i,j]= q1[k,column]
      column+=1
  return Pk2

In [20]:
def findQ2(v:int,m:int,Q1:np.ndarray,T:np.ndarray)->galois.GF2:
  """An implementation of the findQ2 algorithm of the LUOV specification.

  Parameters
  ----------
  v : int
    The v parameter of the LUOV specification.
  m : int
    The m parameter of the LUOV specification.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.
  t : np.ndarray
    A v-by-m matrix.

  Returns
  -------
  np.ndarray
      The second part of Macaulay matrix for quadratic part of P.
  """
  # we use galois to prevent the results of the operations from
  # being out of the finite field
  GF = galois.GF(2)
  T = GF(T)

  Q2 = GF(np.zeros([m,int(m*(m+1)/2)],dtype=np.int8))
  for k in range(m):
    Pk1 = GF(findPk1(v,m,k,Q1))
    Pk2 = GF(findPk2(v,m,k,Q1))
    Pk3 = -np.transpose(T)@Pk1@T+np.transpose(T)@Pk2
    column = 0
    for i in range(m):
      Q2[k,column]= Pk3[i,i]
      column += 1
      for j in range(i+1,m):
        Q2[k,column] = Pk3[i,j] + Pk3[j,i]
        column+=1
  return Q2

# Generate the public key

In [17]:
def keyGen(v:int,m:int,lvl:int,private_seed:bytes)->bytes:
  private_sponge = create_private_sponge(private_seed,v,m,lvl)
  public_seed = get_public_seed(private_sponge)

  T_base = private_sponge[32:]
  T = tMatrixGenerator(T_base,v,m)

  C,L,Q1 = G(v,m,lvl,public_seed)
  Q2 = findQ2(v,m,Q1,T)

  # The first 32 bytes of the public key come from the public seed
  public_key = public_seed

  # The columns of Q2 are concatenated to get a sequence of bits for the
  # public key
  Q2_bits = Q2.flatten('F')
  remaining_bits = len(Q2_bits)
  Q2_bytes = b''

  while(remaining_bits>8):
    byte = ''
    for i in Q2_bits[:8]:
      # Reverse the order of bits
      byte = str(Q2_bits[i]) + byte
    # Turn the bits into a byte and concatenate
    Q2_bytes+=int(byte,2).to_bytes(1,byteorder='big')
    remaining_bits -= 8
    Q2_bits = Q2_bits[8:]

  # If there are remaining bits, we add them to the last byte
  if (remaining_bits>0):
    last_byte = ''
    for bit in Q2_bits:
      # We reverse the order of the bits for the byte
      last_byte = str(bit) + last_byte

  # Add the last byte
  Q2_bytes+=int(last_byte,2).to_bytes(1,byteorder='big')
  # Add the bytes to the public key
  public_key += Q2_bytes
  return public_key

In [53]:
private_seed_test = generate_private_seed()
public_key_test = keyGen(197,57,1,private_seed_test)
print(public_key_test)
print(len(public_key_test))

b'c#g\xed\xbe=\xf6\x1fK\x82\xb5\xc9\xdc\x14\xf7Ct\x06\x07\xbb,\xbaK\xc1\xa6\xcc\xcds\xd9P\xc4\xf1\x00\x00\xb2\xde\xff\x06\xff\xff\x00J\xffR^\x00\xff\x00\xb2\x00\xff:\x1e\x00\x00\xff\xff\x00\xff\xd2\xff\xea\xda\xc6v\xeej\xaa\xff\x0e&\x00\xa2\x00z\xbe\x00\xf2\x86\xf6&\x00\x00\xff\x9e\x16\x8evz\x96\x00\xff\x00\x9a\x00\x00\xea\x00\xff\xce\xba\xff\xfe\x00\x1a\x00\xd2\xff\xce\xff\x00\x9e\x00N\x00\xfa6\xb6\xff\xaa\xff2\x00J\xd2\xe6r\x00\x00\x92\x00\xc6^\x00\x00^\x00\xff\xa2\x00\xa2j>\xf6Z\x00\xff\x16\x8e\xff\xb2\xff&*\x00*\xff\x00\xff\xb2\xf66\xd2\x02\x00\x06\xc2\xff\xff\x1e\x00\xda\x00N\x16\xffV\x06\xffZ\x00\xce\xffF\x00\xe6\x00Z\xc2V\x00\xbe\xce\x00\xf6\x00\x00\x00R\x9a\xff\x00\xff\x00\xff\xff\xff\x00F\xff\x006b\xff\x00\xff\x8en\x00\x00J\xce\x00\x00\xb6\xff\n\xff\x822\x00\x00"\x00\xff&Zb\x00\x00\xff\x06\xff\x00\xaa\xff\x00\xfaJ\xff\x00\x00\x16^\xe2\x0eR\xffn\x00\xff\x00\xff\x82"\x1a\xff\xff26\xba\xffb\x00\x00\x00\x00*\xa2~\x8e\x00\xff\xee\xce\x00\xd2\x82F\xbeB\x8a\x00\x1a\xff\x00\xff\xff\x1

# Sign

## Generate Salt

In [62]:
def generateSalt()->bytes:
  return secrets.token_bytes(16)

## hash digest

In [63]:
def int8_to_binString(intByte:int)->str:
  """Get the 8 bit representation of an integer as a binary string
     without the '0b' indicator.

  The function takes one integer byte, turns it to binary
  and returns an string containing the 8 bit representation of it.

  Parameters
  ----------
  intByte : int
    The byte, labeled as an int by python.

  Returns
  -------
  str
      A string with 8 bits to represent the integer.
  """
  binary = bin(intByte)[2:]
  return binary.zfill(8)

In [64]:
def generate_hash_digest(message:str,salt:bytes,m:int,lvl:int,r:int)->bytes:
  if lvl == 1:
    h_digest_shake = hashlib.shake_128(message+b'\x00'+salt)
  else:
    h_digest_shake = hashlib.shake_256(message+b'\x00'+salt)

  h_digest_bytes = h_digest_shake.digest(math.ceil(m*r/8))

  # Join all the bytes into a single bit-string representation.
  h_digest_bits = ''.join([int8_to_binString(i) for i in h_digest_bytes])

  # Take r bits sequentialy from the string m times, to create a vector within the F(2**r) field
  # The bits that remain after this are discarded.
  h_digest = [int(h_digest_bits[i*r:i*r+r],base=2) for i in range(m)]

  return h_digest


## Generate v

In [65]:
def generateV(r:int, v:int)->list:
  """Generate the vinegar variables vector.

  The function generates the closest amount of bytes to contain r*v bits in order
  to be able to get a vector of size v, with each element having r bits.

  Parameters
  ----------
  r : int
    The r paramter of the LUOV specification.

  v : int
    The v paramter of the LUOV specification.

  Returns
  -------
  list
      A list that represents the vinegar variables.
  """
  vinegar_bytes = secrets.token_bytes(math.ceil(r*v/8))

  # Join all the bytes into a single bit-string representation.
  vinegar_bits = ''.join([int8_to_binString(i) for i in vinegar_bytes])

  # Take r bits sequentialy from the string v times, to create a vector within the F(2**r) field
  # The bits that remain after this are discarded.
  vinegar = [int(vinegar_bits[i*r:i*r+r],base=2) for i in range(v)]
  return vinegar

## Build augmented matrix

In [66]:
def buildAugmentedMatrix(C:galois.FieldArray,
                         L:galois.FieldArray,
                         Q1:galois.FieldArray,
                         T:galois.FieldArray,
                         h:galois.FieldArray,
                         v_array:galois.FieldArray,
                         m:int,v:int,r:int
                         )->tuple[galois.FieldArray,galois.FieldArray]:
  """The implementation of the buildAugmentedMatrix algorithm of the LUOV specification.

  Parameters
  ----------
  C : galois.FieldArray
    The C vector of the LUOV specification.

  L : galois.FieldArray
    The L matrix of the LUOV specification.

  Q1 : galois.FieldArray
    The Q1 matrix of the LUOV specification.

  T : galois.FieldArray
    The T matrix of the LUOV specification.

  h : galois.FieldArray
    The hash digest to target.

  v_array : galois.FieldArray
    A vector containing the vinegar variables.

  m : int
    The m paramter of the LUOV specification.

  v : int
    The v paramter of the LUOV specification.

  r : int
    The r paramter of the LUOV specification.

  Returns
  -------
  (galois.FieldArray,galois.FieldArray)
      The arrays representing the augmented matrix.
      The first being the left part and the second being the right part.
  """
  n = m+v
  v_with_zeros = v_array.copy()
  v_with_zeros.resize(n)
  RHS = h - C-L@v_with_zeros
  T_with_ones = np.concatenate((T,np.identity(m)),axis=0)
  LHS = L@-T_with_ones

  GF = galois.GF(2**r)

  for k in range(m):
    Pk1=GF(findPk1(v,m,k,Q1))
    Pk2=GF(findPk2(v,m,k,Q1))
    RHS[k] = RHS[k] - v_array.T@Pk1@v_array

    Fk2=-(Pk1+Pk1.T)@T+Pk2

    LHS[k] = LHS[k] + v_array@Fk2
  # Here the implementation differs from the specification,
  # instead of returning the augmented matrix LHS and RHS are
  # returned separately to use numpy's solving function
  return LHS,RHS

  # return np.c_[LHS,RHS]


## Sign

In [67]:
def encode_field_element(element:galois.typing.DTypeLike,r:int)->str:
  """Get the r bit representation of an field element as a binary string
     without the '0b' indicator.

  The function takes one field element, turns it to binary
  and returns an string containing the r bit representation of it.

  Parameters
  ----------
  element : galois.typing.DTypeLike
    The field element (The number if you prefer).

  r : int
    The ammount of bits required to encode the element.

  Returns
  -------
  str
      A string with r bits to represent the field element.
  """
  return bin(element)[2:].zfill(r)

In [68]:
def encode_siganture(signature:galois.FieldArray,salt:bytes,n:int,r:int)->bytes:
  """Encoding of the signature with the salt following the LUOV specification.

  The function takes a FieldArray, converts all its elements to binary and
  concatenates them in order, then it encodes the result into bytes (padding with
  zeros at the end to achieve the number of bytes). Finaly, the result is concatenated
  with the salt.

  Parameters
  ----------
  signature : galois.FieldArray
    The FieldArray containing the elements of the signature.

  salt : bytes
    The salt that will be added at the end of the signature.

  n : int
    The number of elements in the signature.

  r : int
    The ammount of bits of each element of the signature.

  Returns
  -------
  bytes
      The bytes of the signature and the salt concatenated.
  """
  binary_signature = ''
  # Turn all the elements in the signature vecto to an r-bit per element
  # binary string
  for i in signature:
    binary_signature += encode_field_element(i,r)
  needed_bits = n*r
  byte_signature = b''
  # Iterate through the binary string, taking 8 bits at a time to get the bytes
  # of the signature
  while needed_bits>8:
    byte_signature += int(binary_signature[:8],2).to_bytes(1,byteorder='big')
    binary_signature = binary_signature[8:]
    needed_bits -= 8
  # The remaining bits are padded with zeros at the end to make the last bytes
  if needed_bits>0:
    for i in range(8-needed_bits):
      binary_signature += '0'
    byte_signature += int(binary_signature,2).to_bytes(1,byteorder='big')
  return byte_signature + salt

In [69]:
def sign(message:str,private_seed:bytes,v:int,m:int,r:int,lvl:int)->bytes:
  private_sponge = create_private_sponge(private_seed,v,m,lvl)
  public_seed = get_public_seed(private_sponge)
  T_base = private_sponge[32:]
  T = tMatrixGenerator(T_base,v,m)

  GF = galois.GF(2**r)

  T = GF(T)

  C,L,Q1 = G(v,m,lvl,public_seed)

  C = GF(C)
  L = GF(L)
  Q1 = GF(Q1)

  salt = generateSalt()
  h = GF(generate_hash_digest(message.encode("utf-8"),salt,m,lvl,r))

  n = m+v
  no_solution = True
  s_prime = np.zeros([m,1])
  while no_solution:
    vinegar = GF(generateV(r=r,v=v))
    A0,A1= buildAugmentedMatrix(C,L,Q1,T,h,vinegar,m,v,r)

    try:
      # Try to solve the equation system given the vinegar variables
      oil = np.linalg.solve(A0,A1)
      # If we find a solution we assume it's unique and end the loop.
      no_solution = False
      # The solution (oil) is concatenated to de vinegar variables vector.
      s_prime = np.concatenate([vinegar,oil])
    except np.linalg.LinAlgError:
      # If no solution is found we stay in the loop to try with new
      # vinegar variables
      no_solution = True

  # Build the matrix for the solution operation
  solution_operand_left = np.concatenate([np.identity(v),np.zeros([m,v])],axis=0)
  solution_operand_right = np.concatenate([-T,np.identity(m)],axis=0)
  solution_operand = np.concatenate([solution_operand_left,solution_operand_right],axis=1)

  # Build S using the matrix solution_operand
  s = GF(solution_operand)@s_prime
  encoded_signature=encode_siganture(s,salt,n,r)
  return encoded_signature

In [72]:
test_sign = sign("Hello World!",private_seed_test,197,57,7,1)
print(test_sign)
print(len(test_sign))

b'<3I\xb9\xfa3\x8d\xc8\x07\x93\xe5\x07\xa3\xcd\x8d@\xacK\x05p\xe6\xe9l`\x05\x0eg5\xbfm\x08C3\xbb\x11\x9c\xc4-\xf6\xef\x01\xe2\xf5b\xad\x8dt,&C\x98s\x82\x94\xac\x08A\x12u\xab\xe9\x04\xabUL\xc0\xca(\xa8q\x0b\x92}\xcaJ\xdcY]\xbcA\xb1\xf1@\xff\xe5\xe9\x87[8[{\xa4W\x1d\xb4\x84\x923\x84<\xeb\x84\xaf?\xf9\x83,\xcc\xef\xeb5=z\xc0\xef\x97M\xff\xafM\xd0\x81\xa5\xea%\xd0\x9eto5J\x8b\x9d\xca\xc9\xfc\xd0\xac\x93\xf8\x1b!x\xf7\x99\xda\xc2\xb11w\xa3\x8c\\\xbdc5\xbdx\xff\xd3\r\xabR\x024\xf5\x97\xc7\xf2o;\xf5\xb6\x15\xd5\xa9\xc2\xa7W\xc5\x8d\xde\x96\x81\x95K\xc2\x8f\n\xcep\xe5\xe0m\xf3S\x05\x9eKD\x16\x80>-f\xe7\x1d\xf7\x8a\x8d\x17N7\xda<)\xd2u\xc8\x13\xd5\x8c\x80AN\x01\xb6\x8f\xdeQR\xd2b\x8a+0I\x12*'
239


# Verify

In [71]:
def evaluatePublicMap(public_key:bytes,s:bytes):
  pass